In [ ]:
import random

In [ ]:
import torch

from torchtext.datasets import TranslationDataset
from torchtext.data import Field

import spacy
import numpy as np

spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

import pandas as pd

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
#Input path of the data to make the noisy data from.
ip_path = ''

#Provide the path where you want to store the noisy data.
op_path_del_gen = ""
op_path_rep_gen = ""
op_path_del_polr = ""
op_path_rep_polr = ""

gen_prob = # <please provide the probability> #0.1
polr_prob = # <please provide the probability> #0.8
#permu_change = 3


src_df = pd.read_csv('trns_polr_dict/amzn_style/trns_polr_src.csv')
src_dict = pd.Series(src_df.is_polarize.values,index=src_df.german_tokens).to_dict()
# trg_df = pd.read_csv('trns_polr_dict/wmt/trns_polr_trg.csv')
# trg_dict = pd.Series(trg_df.is_polarize.values,index=trg_df.english_tokens).to_dict()
del(src_df)
#del(trg_df)

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_de,
                lower = True,
                batch_first = True,
                fix_length=100)

TRG = Field(tokenize = tokenize_en,
            lower = True,
            batch_first = True,
            fix_length=100)

In [ ]:
def pytorch_text_read(ip_path):
    data = TranslationDataset(
        path=ip_path,
        exts=('.de', '.en'),
        fields=(SRC, TRG),
    )
    print(f"Path: {ip_path}, Number of examples: {len(data.examples)}", flush=True)
    return data

In [ ]:
def del_gen_token_randomly(tokens, prob):
    results = [token for token in tokens if not random.random() < prob]
    return " ".join(results)

In [ ]:
def rep_gen_token_randomly(tokens, prob, filler='<style>'):
    tokens_copy = tokens[:]
    for idx in range(len(tokens_copy)):
        if random.random() < prob:
            tokens_copy[idx] = filler
    return " ".join(tokens_copy)

In [ ]:
def del_polr_token_randomly(tokens, prob, dct):
    results = [token for token in tokens if not random.random() < prob and dct.get(token, False)== True]
    return " ".join(results)

In [ ]:
def rep_polr_token_randomly(tokens, prob, dct, filler='<style>'):
    tokens_copy = tokens[:]
    for idx in range(len(tokens_copy)):
        if random.random() < prob and dct.get(tokens_copy[idx], False)== True:
            tokens_copy[idx] = filler
    return " ".join(tokens_copy)

In [ ]:
# def random_token_permutation(tokens, permu_range):
#     new_indices = [i+random.uniform(0, permu_range+1) for i in range(len(tokens))]
#     res = [x for _, x in sorted(zip(new_indices, tokens), key=lambda pair: pair[0])]
#     return " ".join(res)

In [ ]:
data = pytorch_text_read(ip_path)
for i in range(50):
    with open(op_path_del_gen+str(i)+".de", 'w') as op_del_gen, \
            open(op_path_rep_gen+str(i)+".de", 'w') as op_rep_gen, \
            open(op_path_del_polr+str(i)+".de", 'w') as op_del_polr, \
            open(op_path_rep_polr+str(i)+".de", 'w') as op_rep_polr:

            for idx in range(len(data.examples)):
                de = vars(data.examples[idx])['src']
                #en = vars(data.examples[idx])['trg']

                del_tokens = del_gen_token_randomly(de, gen_prob)
                if(del_tokens == ''):
                    del_tokens = " ".join(de)
                op_del_gen.write(del_tokens + '\n')

                rep_tokens = rep_gen_token_randomly(de, gen_prob)
                op_rep_gen.write(rep_tokens + '\n')

                del_polr_tokens = del_polr_token_randomly(de, polr_prob, src_dict)
                if (del_polr_tokens == ''):
                    del_polr_tokens = " ".join(de)
                op_del_polr.write(del_polr_tokens + '\n')

                rep_polr_tokens = rep_polr_token_randomly(de, polr_prob, src_dict)
                op_rep_polr.write(rep_polr_tokens + '\n')